# Creating FABnet IPv*Ext Network

FABRIC provides a pair of layer 3 IP networking services across every FABRIC site (FABnetv4 and FABnetv6). This notebook describes how to enable external connectivity on FabNet services.

## Import the FABlib Library


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

try:
    fablib = fablib_manager()
                     
    fablib.show_config();
except Exception as e:
    print(f"Exception: {e}")

# Load the Plugins

In [ ]:
from plugins import Plugins
import traceback
try:
    Plugins.load()
except Exception as e:
    traceback.print_exc()

## Create the Experiment Slice

The following creates two nodes, on different sites, with basic NICs one connected to FabNetV4Ext and other connected to FabNetV6Ext.
It then enables/disables external connectivity on FabNetV*Ext. 

NIC component models options:
- NIC_Basic: 100 Gbps Mellanox ConnectX-6 SR-IOV VF (1 Port)
- NIC_ConnectX_5: 25 Gbps Dedicated Mellanox ConnectX-5 PCI Device (2 Ports) 
- NIC_ConnectX_6: 100 Gbps Dedicated Mellanox ConnectX-6 PCI Device (2 Ports) 

In [ ]:
slice_name = 'MySlice2-ext-2'
[site1,site2] = fablib.get_random_sites(count=2)
site1=site2='UKY'
print(f"Sites: {site1}, {site2}")

node1_name = 'Node1'
node2_name = 'Node2'

network1_name='net1'
network2_name='net2'

node1_nic_name = 'nic1'
node2_nic_name = 'nic2'

In [ ]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    # Node1
    node1 = slice.add_node(name=node1_name, site=site1)
    iface1 = node1.add_component(model='NIC_Basic', name=node1_nic_name).get_interfaces()[0]
    
    # Node2
    node2 = slice.add_node(name=node2_name, site=site2)
    iface2  = node2.add_component(model='NIC_Basic', name=node2_nic_name).get_interfaces()[0]
    
    # NetworkS
    net1 = slice.add_l3network(name=network1_name, interfaces=[iface1], type='IPv6Ext')
    net2 = slice.add_l3network(name=network2_name, interfaces=[iface2], type='IPv4Ext')
    
    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")

## Observe the Slice's Attributes

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.show()
    slice.list_nodes()
    slice.list_networks()
    slice.list_interfaces()
except Exception as e:
    print(f"Exception: {e}")

# Update Network Service - Enable/Disable Public IP Addresses

In [ ]:
try:
    network1 = slice.get_network(name=network1_name)
    network1_available_ips = network1.get_available_ips()
    
    # Enable Public IPv6 make_ip_publicly_routable
    network1.change_public_ip(ipv6=[str(network1_available_ips[0])])
    
    # Uncomment to Disable Public IPv6
    #network1.update_labels(ipv6_subnet=[])
    
    network2 = slice.get_network(name=network2_name)
    network2_available_ips = network2.get_available_ips()
    
    # Enable Public IPv4
    # Due to limited IPv4 address space, we use a single subnet for all the slices for a specific site
    # Please note the first IP on the subnet is requested to be made public. If the IP is already in use, ControlFramework assigns an available IP 
    # and saves in the Labels. If no available IP is found, an error is returned.
    network2.change_public_ip(ipv4=[str(network2_available_ips[0])])
    
    # Uncomment to Disable Public IPv4
    #network2.update_labels(ipv4_subnet=[])
    slice.submit()
    
except Exception as e:
    print(f"Exception: {e}")
    import traceback
    traceback.print_exc()

##  Configure IP Addresses

### Configure Node1 - FABnetv6

Configure the interface on node1.

In [ ]:
try:
    # Get Node
    node1 = slice.get_node(name=node1_name)        
    # Get Network
    node1_network = slice.get_network(name=network1_name) 
    # Get Interface
    node1_iface = node1.get_interface(network_name=network1_name)  
    
    # Configure first IP from the subnet
    node1_addr = network1_available_ips.pop(0)
    node1_iface.ip_addr_add(addr=node1_addr, subnet=network1.get_subnet())
        
    stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')
    stdout, stderr = node1.execute(f'ip route list')
except Exception as e:
    print(f"Exception: {e}")

### Configure Node2 - FABnetv4

Always use the assigned IP from the Network.Labels as user requested IP might have been updated by ControlFramework. 

NOTE: User is responsible to add the route rules controlling which external networks are allowed to access. This route should not be made default route ensuring the management traffic is not disrupted or misdirected.



In [ ]:
try:
    # Get Node
    node2 = slice.get_node(name=node2_name)      
    # Get Network
    node2_network = slice.get_network(name=network2_name) 
    # Get Interface
    node2_iface = node2.get_interface(network_name=network2_name) 
    
    # Get Assigned Public IP returned by Control Framework
    node2_addr = node2_network.get_fim_network_service().labels.ipv4[0]
    
    # Configure the assigned public IP
    node2_iface.ip_addr_add(addr=node2_addr, subnet=network2.get_subnet())
        
    stdout, stderr = node2.execute(f'ip addr show {node2_iface.get_os_interface()}')
    stdout, stderr = node2.execute(f'ip route list')
except Exception as e:
    print(f"Exception: {e}")

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")